In [1]:
import numpy as np

def _l2(a, b, axis=-1, eps=1e-9):
    """Euclidean distance with tiny floor for stability."""
    return np.sqrt(np.maximum(np.sum((a - b) ** 2, axis=axis), eps))

def ade_single(y_true: np.ndarray,
               y_pred: np.ndarray,
               mask: np.ndarray | None = None) -> float:
    """
    Average Displacement Error for a single predicted path.

    Args:
        y_true: (T, D) or (N, T, D)
        y_pred: (T, D) or (N, T, D) — same shape as y_true
        mask:   optional (T,) or (N, T) boolean/0-1 (valid timesteps)

    Returns:
        Scalar ADE (float).
    """
    # Normalize shapes to (N, T, D)
    if y_true.ndim == 2:
        y_true = y_true[None, ...]
        y_pred = y_pred[None, ...]
        if mask is not None and mask.ndim == 1:
            mask = mask[None, ...]
    elif y_true.ndim != 3:
        raise ValueError("y_true must be (T, D) or (N, T, D)")

    if y_pred.shape != y_true.shape:
        raise ValueError("y_pred must have the same shape as y_true")

    N, T, _ = y_true.shape
    d = _l2(y_pred, y_true, axis=-1)  # (N, T)

    if mask is not None:
        if mask.shape == (T,):
            mask = mask[None, :]
        if mask.shape != (N, T):
            raise ValueError("mask must be (T,) or (N, T)")
        d = d * mask
        denom = np.clip(mask.sum(axis=1), 1e-9, None)  # per-sample valid steps
    else:
        denom = np.full(N, T, dtype=float)

    per_sample = d.sum(axis=1) / denom
    return float(per_sample.mean())

def fde_single(y_true: np.ndarray,
               y_pred: np.ndarray) -> float:
    """
    Final Displacement Error for a single predicted path.

    Args:
        y_true: (T, D) or (N, T, D)
        y_pred: same shape as y_true

    Returns:
        Scalar FDE (float).
    """
    # Normalize to (N, T, D)
    if y_true.ndim == 2:
        y_true = y_true[None, ...]
        y_pred = y_pred[None, ...]
    elif y_true.ndim != 3:
        raise ValueError("y_true must be (T, D) or (N, T, D)")

    gt_last = y_true[:, -1, :]  # (N, D)
    pr_last = y_pred[:, -1, :]  # (N, D)
    per_sample = _l2(pr_last, gt_last, axis=-1)  # (N,)
    return float(per_sample.mean())


In [43]:
out = [
    (646.9031108833622, 624.1285402212566),
    (646.5865840464116, 624.6741305105683),
    (646.2700572094611, 625.21972079988),
    (645.9535303725106, 625.7653110891916),
    (645.6370035355601, 626.3109013785033),
    (645.3204766986096, 626.856491667815),
    (645.0039498616591, 627.4020819571267),
    (644.6874230247086, 627.9476722464384),
    (644.370896187758, 628.4932625357501),
    (644.0543693508075, 629.0388528250618)
]

y_true =[(686.0, 646.0), (685.7521057128906, 646.0467834472656), (685.333984375, 646.0319519042969), (684.8723449707031, 645.7224426269531), (684.6224060058594, 645.4742126464844), (684.2150573730469, 645.1096496582031), (684.2150573730469, 645.1096496582031), (683.6287231445312, 644.8713684082031), (682.3135375976562, 644.2965087890625), (681.9295349121094, 644.5404968261719)]















In [44]:
y_true = np.array(y_true)
y_pred = np.array(out)

# Optionally ignore the last 2 timesteps:
# mask = np.ones(T, dtype=float); mask[-2:] = 0.0

print("ADE:", ade_single(y_true, y_pred, ))
print("FDE:", fde_single(y_true, y_pred))

ADE: 43.13320028980497
FDE: 40.924676333884165
